In [1]:
## Importing Dependencies
 
import pandas as pd
import requests
import json
import sys
from pymongo import MongoClient
from pprint import pprint
from sqlalchemy import create_engine
sys.path.append('/Users/tokar/UNC_DA/MY_WORK/Personal_Code')
from my_api_config import ticket_master_key
import time

In [2]:
# Function to remove unnecessary keys from the JSON responses
def remove_keys(response):
    if '_embedded' in response:  # Check if '_embedded' key exists
        # Defining keys to remove in each of their various sections
        keys_to_remove_events = ["test", "locale", "images", "sales", "pleaseNote","info","outlets", "promoter", "promoters", "products", "seatmap", "accessibility", "ticketLimit", "ageRestrictions", "ticketing", "_links"]
        keys_to_remove_dates = ["timezone", "spanMultipleDays", "status"]
        keys_to_remove_dates_start = ["dateTBD", "dateTBA", "timeTBA", "noSpecificTime"]
        keys_to_remove_embedded_venues = ["test", "url", "locale", "aliases", "images", "dmas", "generalInfo", "upcomingEvents", "_links", "social", "boxOfficeInfo", "parkingDetail", "accessibleSeatingDetail"]
        keys_to_remove_embedded_attractions = ["test", "locale", "externalLinks", "aliases", "images", "upcomingEvents", "_links"]
        keys_to_remove_top = ["_links", "page"]

        # Remove top-level keys
        for key in keys_to_remove_top:
            if key in response:
                del response[key]

        # Remove keys from each event
        for event in response.get("_embedded", {}).get("events", []):
            # Remove keys from "events" section
            for key in keys_to_remove_events:
                if key in event:
                    del event[key]
            # Remove keys from "dates" section
            if 'dates' in event:
                for key in keys_to_remove_dates:
                    if key in event['dates']:
                        del event['dates'][key]
                # Remove keys from "start" section within "dates"
                if 'start' in event['dates']:
                    for key in keys_to_remove_dates_start:
                        if key in event['dates']['start']:
                            del event['dates']['start'][key]
            # Remove keys from "venues" section
            if '_embedded' in event:
                for venue in event['_embedded'].get('venues', []):
                    for key in keys_to_remove_embedded_venues:
                        if key in venue:
                            del venue[key]
            # Remove keys from "attractions" section
            if '_embedded' in event and 'attractions' in event['_embedded']:
                for attraction in event['_embedded']['attractions']:
                    for key in keys_to_remove_embedded_attractions:
                        if key in attraction:
                            del attraction[key]

In [4]:

# Loop for making 1000 API Calls

# Establishing MongoDB Connection
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["ticket_master_db"]
events_collection = db["events"]

# Iterating through 1000 API calls
for page_number in range(0, 100):  # Adjust the range as needed
    
    # Establishing the URL of the API call
    url_base = "https://app.ticketmaster.com/discovery/v2/events.json?"
    url_country = "&countryCode=US"
    url_source = "&source=ticketmaster"
    url_segment = "&segmentName=music"
    url_size = "&size=200"
    url_date_start = "&startDateTime=2024-04-05T00:00:00Z"
    url_date_end = "&endDateTime=2024-10-01T00:00:00Z"
    url_sort = "&sort=name,date,desc"
    url_page = "&page=" + str(page_number)
    query_url = url_base + "apikey=" + ticket_master_key + url_source + url_date_start + url_date_end + url_country + url_segment + url_size + url_page + url_sort
    
    # Storing API call response into JSON
    response = requests.get(query_url).json()
    
    # Check if '_embedded' key exists
    if '_embedded' in response:
        # Running function to remove unnecessary fields
        remove_keys(response)
        
        # Extract the events array if present
        events = response["_embedded"].get("events", [])
        
        # Insert each event into the MongoDB collection
        for event in events:
            events_collection.insert_one(event)
    
    time.sleep(4)  # Add a delay if needed

In [7]:
client = MongoClient("mongodb://localhost:27017/")
db = client["ticket_master_db"]
events_collection = db["events"]

# Query the data you want to export
data = list(events_collection.find({}))

# Convert ObjectId to string
for item in data:
    item["_id"] = str(item["_id"])

# Serialize the data into JSON format
ticketmaster_data = json.dumps(data, indent=4)

# Write the JSON data to a file
with open("ticket_master.json", "w") as file:
    file.write(ticketmaster_data)

print("Data exported to ticket_master.json")

Data exported to ticket_master.json
